In [3]:
import pandas as pd
import numpy as np
import re
pd.options.mode.chained_assignment = None  # default='warn'

In [96]:
%reset_selective -f all

In [97]:
starbucks_dirty = pd.read_csv("startbucks_Jeffrey.csv")
starbucks_dirty

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
0,4712,49987-272024,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",hyatt-bethesda-metro-1-bethesda-metro-center-b...,38.984316,-77.094665,1 Bethesda Metro Center,NaN,NaN,Bethesda,MD,208145325,-240,Eastern Standard Time,GMT-05:00 America/New_York
1,4713,743-294452,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",7200-wisconsin-ave-7200-wisconsin-ave-bethesda...,38.981224,-77.093287,7200 Wisconsin Ave,120,NaN,Bethesda,MD,20814,-240,Eastern Standard Time,GMT-05:00 America/New_York
2,4714,13235-105606,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",van-ness-udc-4250-connecticut-ave-nw-van-ness-...,38.944498,-77.064207,"4250 Connecticut Ave., NW",NaN,Van Ness Metro Building,Washington,DC,200081173,-240,Eastern Standard Time,GMT-05:00 America/New_York
3,4715,47040-256424,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",dakota-crossing-2430-market-street-ne-washingt...,38.920244,-76.952774,2430 Market Street NE,1001,NaN,Washington,DC,200183852,-240,Eastern Standard Time,GMT-05:00 America/New_York
4,4716,7764-61020,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",friendship-heights-5454-wisconsin-avenue-the-b...,38.962553,-77.086862,5454 Wisconsin Avenue,NaN,The Barlow Building,Chevy Chase,MD,208156901,-240,Eastern Standard Time,GMT-05:00 America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,17639,21472-213065,GB,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 7:0...",rhyl-kinmel-park-eastbound-kinmel-park-a-55-ea...,53.270439,-3.519619,"Kinmel Park, A55 Eastbound",Bodelwyddon,NaN,Rhyl,WLS,LL18 5XW,0,GMT Standard Time,GMT+00:00 Europe/London
3794,17640,47867-260072,GB,FR,"[{'dayName': 'Today', 'hours': '6:30 AM to 5:3...",welwyn-garden-city-howard-centre-unit-56-welwy...,51.802087,-0.204220,Unit 56,The Howard Centre,NaN,Welwyn Garden City,ENG,AL8 6HA,0,GMT Standard Time,GMT+00:00 Europe/London
3795,17641,9139-152280,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",birchanger-welcome-break-m-11-motorway-junctio...,51.871540,0.194940,"M11 Motorway, Junction 8","Old Dunmow Road, Bishop Stortford",NaN,Hertfordshire,ENG,CM23 5QZ,0,GMT Standard Time,GMT+00:00 Europe/London
3796,17642,27642-248279,GB,FR,"[{'dayName': 'Today', 'hours': '7:00 AM to 6:0...",bishops-stortford-jacksons-square-jacksons-squ...,51.870230,0.160930,Jacksons Square,NaN,NaN,Bishop's Stortford,ENG,CM23 3AB,0,GMT Standard Time,GMT+00:00 Europe/London


# cluster by country

In [98]:
dirty_us = starbucks_dirty[starbucks_dirty['countryCode'] == 'US']
dirty_us
# pd.unique(starbucks_dirty['countryCode'])

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
0,4712,49987-272024,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",hyatt-bethesda-metro-1-bethesda-metro-center-b...,38.984316,-77.094665,1 Bethesda Metro Center,NaN,NaN,Bethesda,MD,208145325,-240,Eastern Standard Time,GMT-05:00 America/New_York
1,4713,743-294452,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",7200-wisconsin-ave-7200-wisconsin-ave-bethesda...,38.981224,-77.093287,7200 Wisconsin Ave,120,NaN,Bethesda,MD,20814,-240,Eastern Standard Time,GMT-05:00 America/New_York
2,4714,13235-105606,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",van-ness-udc-4250-connecticut-ave-nw-van-ness-...,38.944498,-77.064207,"4250 Connecticut Ave., NW",NaN,Van Ness Metro Building,Washington,DC,200081173,-240,Eastern Standard Time,GMT-05:00 America/New_York
3,4715,47040-256424,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",dakota-crossing-2430-market-street-ne-washingt...,38.920244,-76.952774,2430 Market Street NE,1001,NaN,Washington,DC,200183852,-240,Eastern Standard Time,GMT-05:00 America/New_York
4,4716,7764-61020,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",friendship-heights-5454-wisconsin-avenue-the-b...,38.962553,-77.086862,5454 Wisconsin Avenue,NaN,The Barlow Building,Chevy Chase,MD,208156901,-240,Eastern Standard Time,GMT-05:00 America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,7530,24426-237911,US,CO,"[{'dayName': 'Today', 'hours': '4:30 AM to 9:0...",27th-avenue-loop-101-phoenix-20216-n-27th-aven...,33.670760,-112.117874,20216 N 27th Avenue,NaN,NaN,Phoenix,AZ,850273207,-420,US Mountain Standard Time,GMT-07:00 America/Phoenix
2819,7531,27544-245014,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 8:0...",target-phoenix-t-1141-2727-w-agua-fria-fwy-pho...,33.667883,-112.118782,2727 W Agua Fria Fwy,NaN,NaN,Phoenix,AZ,85027,-420,US Mountain Standard Time,GMT-07:00 America/Phoenix
2820,7532,8974-95021,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",happy-valley-i-17-phoenix-2501-west-happy-vall...,33.711073,-112.111442,2501 West Happy Valley Road,#50-1200,NaN,Phoenix,AZ,850853701,-420,US Mountain Standard Time,GMT-07:00 America/Phoenix
2821,7533,9065-113971,US,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 1:3...",usaa-phoenix-campus-1-norterra-dr-phoenix-az-8...,33.720295,-112.111798,1 Norterra Dr,NaN,NaN,Phoenix,AZ,850858204,-420,US Mountain Standard Time,GMT-07:00 America/Phoenix


In [99]:
dirty_gb = starbucks_dirty[starbucks_dirty['countryCode'] == 'GB']
dirty_gb
# type(dirty_gb)

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
2823,16669,9155-152277,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-south-welcome-break-m-4-motorway-westb...,51.483556,-1.557143,"M4 Motorway Westbound, Junction 14/15",Lambourne,NaN,Berkshire,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00 Europe/London
2824,16670,22194-218828,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-south-welcome-break-dt-m-4-motorway-j-...,51.482387,-1.555109,M4 Motorway J14 15 membury,NaN,NaN,Hungerford,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00 Europe/London
2825,16671,18362-190424,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-north-welcome-break-dt-m-4-motorway-ju...,51.481264,-1.556526,"M4 Motorway, Junction 14/15",Lambourne,NaN,Hungerford,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00 Europe/London
2826,16672,9136-152279,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-north-welcome-break-m-4-motorway-eastb...,51.481177,-1.557422,M4 Motorway Eastbound Junction 14 15,NaN,NaN,Berkshire,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00 Europe/London
2827,16673,47832-260044,GB,FR,"[{'dayName': 'Today', 'hours': '10:00 AM to 8:...",swindon-great-western-outlet-unit-51-swindon-e...,51.562617,-1.798111,Unit 51,Great Western Designer Village,NaN,Swindon,ENG,SN2 2DZ,0,GMT Standard Time,GMT+00:00 Europe/London
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,17639,21472-213065,GB,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 7:0...",rhyl-kinmel-park-eastbound-kinmel-park-a-55-ea...,53.270439,-3.519619,"Kinmel Park, A55 Eastbound",Bodelwyddon,NaN,Rhyl,WLS,LL18 5XW,0,GMT Standard Time,GMT+00:00 Europe/London
3794,17640,47867-260072,GB,FR,"[{'dayName': 'Today', 'hours': '6:30 AM to 5:3...",welwyn-garden-city-howard-centre-unit-56-welwy...,51.802087,-0.204220,Unit 56,The Howard Centre,NaN,Welwyn Garden City,ENG,AL8 6HA,0,GMT Standard Time,GMT+00:00 Europe/London
3795,17641,9139-152280,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",birchanger-welcome-break-m-11-motorway-junctio...,51.871540,0.194940,"M11 Motorway, Junction 8","Old Dunmow Road, Bishop Stortford",NaN,Hertfordshire,ENG,CM23 5QZ,0,GMT Standard Time,GMT+00:00 Europe/London
3796,17642,27642-248279,GB,FR,"[{'dayName': 'Today', 'hours': '7:00 AM to 6:0...",bishops-stortford-jacksons-square-jacksons-squ...,51.870230,0.160930,Jacksons Square,NaN,NaN,Bishop's Stortford,ENG,CM23 3AB,0,GMT Standard Time,GMT+00:00 Europe/London


# check City, countrySubdivisionCode characters case

In [100]:
dirty_gb['countrySubdivisionCode'] = dirty_gb['countrySubdivisionCode'].str.upper()
dirty_us['countrySubdivisionCode'] = dirty_us['countrySubdivisionCode'].str.upper()
print(pd.unique(dirty_gb['countrySubdivisionCode']))
print(pd.unique(dirty_us['countrySubdivisionCode']))

['ENG' 'WLS' 'SCT']
['MD' 'DC' 'NY' 'OH' 'MI' 'OR' 'TX' 'WA' 'WI' 'AL' 'MS' 'FL' 'CA' 'NM'
 'UT' 'VA' 'CO' 'IN' 'SD' 'IL' 'AZ' 'GA' 'LA' 'ME' 'NE' 'IA' 'OK' 'NV'
 'NJ' 'NC' 'SC' 'TN' 'AK' 'MO' 'MA' 'ID' 'MN' 'CT' 'RI']


In [101]:
dirty_gb['city'] = dirty_gb['city'].str.lower()
dirty_us['city'] = dirty_us['city'].str.lower()
print(pd.unique(dirty_gb['city']))
print(pd.unique(dirty_us['city']))

['berkshire' 'hungerford' 'swindon' 'newbury' 'didcot' 'andover' 'wigan'
 'reading' 'winchester' 'burford' 'headington' 'basingstoke' 'oxford'
 'melksham' 'henley-on-thames' 'bicester' 'trowbridge' 'salisbury' 'thame'
 'wokingham' 'hampshire' 'hants' 'marlow' 'frome' 'fleet' 'birmingham'
 'redditch' 'solihull' 'solihull, west midlands' 'edgbaston' 'quinton'
 'droitwich' 'brierley hill' 'dudley' 'kidderminster' 'warwick' 'walsall'
 'stratford upon avon' 'worcester' 'coventry' 'wolverhampton'
 'leamington spa' 'corley' 'ashorne' 'banbury' 'rugby' 'nuneaton'
 'northampton' 'towcester' 'gloucestershire' 'nr stroud' 'bristol'
 'gloucester' 'monmouth' 'bath' 'ross on wye' 'cheltenham' 'newport'
 'cwmbran' 'tewkesbury' 'weston super mare' 'cardiff' 'warminster'
 'yorkshire' 'brighouse' 'cleckheaton' 'mirfield' 'huddersfield'
 'bradford' 'birstall' 'batley' 'dewsbury' 'leeds' 'west yorkshire'
 'leeds city point' 'wakefield' 'keighley' 'barnsley' 'castleford'
 'rochdale' 'skipton' 'harrogate' '

# split time zone

In [102]:
timezone_gb = dirty_gb.copy()
x = dirty_gb.loc[:,'olsonTimeZoneId'].str.split(' ', expand = True)
timezone_gb['GMT'] = x[0]
timezone_gb['GMT_region'] = x[1]
timezone_gb.drop(labels=['olsonTimeZoneId'],axis='columns', inplace = True)
timezone_gb


,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,GMT,GMT_region
2823,16669,9155-152277,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-south-welcome-break-m-4-motorway-westb...,51.483556,-1.557143,"M4 Motorway Westbound, Junction 14/15",Lambourne,NaN,berkshire,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00,Europe/London
2824,16670,22194-218828,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-south-welcome-break-dt-m-4-motorway-j-...,51.482387,-1.555109,M4 Motorway J14 15 membury,NaN,NaN,hungerford,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00,Europe/London
2825,16671,18362-190424,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-north-welcome-break-dt-m-4-motorway-ju...,51.481264,-1.556526,"M4 Motorway, Junction 14/15",Lambourne,NaN,hungerford,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00,Europe/London
2826,16672,9136-152279,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-north-welcome-break-m-4-motorway-eastb...,51.481177,-1.557422,M4 Motorway Eastbound Junction 14 15,NaN,NaN,berkshire,ENG,RG17 7TZ,0,GMT Standard Time,GMT+00:00,Europe/London
2827,16673,47832-260044,GB,FR,"[{'dayName': 'Today', 'hours': '10:00 AM to 8:...",swindon-great-western-outlet-unit-51-swindon-e...,51.562617,-1.798111,Unit 51,Great Western Designer Village,NaN,swindon,ENG,SN2 2DZ,0,GMT Standard Time,GMT+00:00,Europe/London
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,17639,21472-213065,GB,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 7:0...",rhyl-kinmel-park-eastbound-kinmel-park-a-55-ea...,53.270439,-3.519619,"Kinmel Park, A55 Eastbound",Bodelwyddon,NaN,rhyl,WLS,LL18 5XW,0,GMT Standard Time,GMT+00:00,Europe/London
3794,17640,47867-260072,GB,FR,"[{'dayName': 'Today', 'hours': '6:30 AM to 5:3...",welwyn-garden-city-howard-centre-unit-56-welwy...,51.802087,-0.204220,Unit 56,The Howard Centre,NaN,welwyn garden city,ENG,AL8 6HA,0,GMT Standard Time,GMT+00:00,Europe/London
3795,17641,9139-152280,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",birchanger-welcome-break-m-11-motorway-junctio...,51.871540,0.194940,"M11 Motorway, Junction 8","Old Dunmow Road, Bishop Stortford",NaN,hertfordshire,ENG,CM23 5QZ,0,GMT Standard Time,GMT+00:00,Europe/London
3796,17642,27642-248279,GB,FR,"[{'dayName': 'Today', 'hours': '7:00 AM to 6:0...",bishops-stortford-jacksons-square-jacksons-squ...,51.870230,0.160930,Jacksons Square,NaN,NaN,bishop's stortford,ENG,CM23 3AB,0,GMT Standard Time,GMT+00:00,Europe/London


In [103]:
timezone_us = dirty_us.copy()
x = dirty_us.loc[:,'olsonTimeZoneId'].str.split(' ', expand = True)
timezone_us['GMT'] = x[0]
timezone_us['GMT_region'] = x[1]
timezone_us.drop(labels=['olsonTimeZoneId'],axis='columns', inplace = True)
x
timezone_us

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,GMT,GMT_region
0,4712,49987-272024,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",hyatt-bethesda-metro-1-bethesda-metro-center-b...,38.984316,-77.094665,1 Bethesda Metro Center,NaN,NaN,bethesda,MD,208145325,-240,Eastern Standard Time,GMT-05:00,America/New_York
1,4713,743-294452,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",7200-wisconsin-ave-7200-wisconsin-ave-bethesda...,38.981224,-77.093287,7200 Wisconsin Ave,120,NaN,bethesda,MD,20814,-240,Eastern Standard Time,GMT-05:00,America/New_York
2,4714,13235-105606,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",van-ness-udc-4250-connecticut-ave-nw-van-ness-...,38.944498,-77.064207,"4250 Connecticut Ave., NW",NaN,Van Ness Metro Building,washington,DC,200081173,-240,Eastern Standard Time,GMT-05:00,America/New_York
3,4715,47040-256424,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",dakota-crossing-2430-market-street-ne-washingt...,38.920244,-76.952774,2430 Market Street NE,1001,NaN,washington,DC,200183852,-240,Eastern Standard Time,GMT-05:00,America/New_York
4,4716,7764-61020,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",friendship-heights-5454-wisconsin-avenue-the-b...,38.962553,-77.086862,5454 Wisconsin Avenue,NaN,The Barlow Building,chevy chase,MD,208156901,-240,Eastern Standard Time,GMT-05:00,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,7530,24426-237911,US,CO,"[{'dayName': 'Today', 'hours': '4:30 AM to 9:0...",27th-avenue-loop-101-phoenix-20216-n-27th-aven...,33.670760,-112.117874,20216 N 27th Avenue,NaN,NaN,phoenix,AZ,850273207,-420,US Mountain Standard Time,GMT-07:00,America/Phoenix
2819,7531,27544-245014,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 8:0...",target-phoenix-t-1141-2727-w-agua-fria-fwy-pho...,33.667883,-112.118782,2727 W Agua Fria Fwy,NaN,NaN,phoenix,AZ,85027,-420,US Mountain Standard Time,GMT-07:00,America/Phoenix
2820,7532,8974-95021,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",happy-valley-i-17-phoenix-2501-west-happy-vall...,33.711073,-112.111442,2501 West Happy Valley Road,#50-1200,NaN,phoenix,AZ,850853701,-420,US Mountain Standard Time,GMT-07:00,America/Phoenix
2821,7533,9065-113971,US,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 1:3...",usaa-phoenix-campus-1-norterra-dr-phoenix-az-8...,33.720295,-112.111798,1 Norterra Dr,NaN,NaN,phoenix,AZ,850858204,-420,US Mountain Standard Time,GMT-07:00,America/Phoenix


# Merge address with "|"

In [104]:
add_time_us = timezone_us.copy()
add_time_us['address'] = add_time_us[['streetAddressLine1', 'streetAddressLine2', 'streetAddressLine3']].astype(str).agg('|'.join, axis=1)
add_time_us.replace( regex=r'\|nan', value='', inplace = True)
add_time_us['address']



0                                 1 Bethesda Metro Center
1                                  7200 Wisconsin Ave|120
2       4250 Connecticut Ave., NW|Van Ness Metro Building
3                              2430 Market Street NE|1001
4               5454 Wisconsin Avenue|The Barlow Building
                              ...                        
2818                                  20216 N 27th Avenue
2819                                 2727 W Agua Fria Fwy
2820                 2501 West Happy Valley Road|#50-1200
2821                                        1 Norterra Dr
2822                             4123 W. Thunderbird Road
Name: address, Length: 2823, dtype: object

In [105]:
add_time_gb = timezone_gb.copy()
add_time_gb['address'] = add_time_gb[['streetAddressLine1', 'streetAddressLine2', 'streetAddressLine3']].astype(str).agg('|'.join, axis=1)
add_time_gb.replace( regex=r'\|nan', value='', inplace = True)
add_time_gb['address']

2823      M4 Motorway Westbound, Junction 14/15|Lambourne
2824                          M4 Motorway J14 15 membury 
2825                M4 Motorway, Junction 14/15|Lambourne
2826                 M4 Motorway Eastbound Junction 14 15
2827               Unit 51|Great Western Designer Village
                              ...                        
3793              Kinmel Park,  A55 Eastbound|Bodelwyddon
3794                            Unit 56|The Howard Centre
3795    M11 Motorway, Junction 8|Old Dunmow Road, Bish...
3796                                      Jacksons Square
3797                     Unit B2, Harvey Shopping Centre,
Name: address, Length: 975, dtype: object

# split schedule

In [106]:
schedule_us = add_time_us['schedule'].to_frame()
schedule_gb = add_time_gb['schedule'].to_frame()
hours_add_time_us = add_time_us.copy()
hours_add_time_gb = add_time_gb.copy()
print(schedule_us)
print(schedule_gb)
# type(schedule_us)
# type(schedule_gb)

                                               schedule
0     [{'dayName': 'Today', 'hours': '5:30 AM to 8:0...
1     [{'dayName': 'Today', 'hours': '5:00 AM to 8:0...
2     [{'dayName': 'Today', 'hours': '5:00 AM to 8:0...
3     [{'dayName': 'Today', 'hours': '5:30 AM to 8:0...
4     [{'dayName': 'Today', 'hours': '5:00 AM to 7:3...
...                                                 ...
2818  [{'dayName': 'Today', 'hours': '4:30 AM to 9:0...
2819  [{'dayName': 'Today', 'hours': '8:00 AM to 8:0...
2820  [{'dayName': 'Today', 'hours': '5:00 AM to 7:3...
2821  [{'dayName': 'Today', 'hours': '7:00 AM to 1:3...
2822  [{'dayName': 'Today', 'hours': '4:30 AM to 11:...

[2823 rows x 1 columns]
                                               schedule
2823  [{'dayName': 'Today', 'hours': '6:00 AM to 10:...
2824  [{'dayName': 'Today', 'hours': '5:30 AM to 8:0...
2825  [{'dayName': 'Today', 'hours': '5:30 AM to 8:0...
2826  [{'dayName': 'Today', 'hours': '6:00 AM to 10:...
2827  [{'dayName': 'Tod

In [107]:
schedule_us_split = schedule_us['schedule'].str.split('}, {', expand = True)
print(schedule_us_split.count)
schedule_gb_split = schedule_gb['schedule'].str.split('}, {', expand = True)
print(schedule_gb_split.count)
# schedule_us_split.fillna('Closed', inplace = True)
# print(schedule_us_split[1][2])

<bound method DataFrame.count of                                                       0  \
0     [{'dayName': 'Today', 'hours': '5:30 AM to 8:0...   
1     [{'dayName': 'Today', 'hours': '5:00 AM to 8:0...   
2     [{'dayName': 'Today', 'hours': '5:00 AM to 8:0...   
3     [{'dayName': 'Today', 'hours': '5:30 AM to 8:0...   
4     [{'dayName': 'Today', 'hours': '5:00 AM to 7:3...   
...                                                 ...   
2818  [{'dayName': 'Today', 'hours': '4:30 AM to 9:0...   
2819  [{'dayName': 'Today', 'hours': '8:00 AM to 8:0...   
2820  [{'dayName': 'Today', 'hours': '5:00 AM to 7:3...   
2821  [{'dayName': 'Today', 'hours': '7:00 AM to 1:3...   
2822  [{'dayName': 'Today', 'hours': '4:30 AM to 11:...   

                                                      1  \
0     'dayName': 'Tomorrow', 'hours': '5:30 AM to 8:...   
1     'dayName': 'Tomorrow', 'hours': '5:00 AM to 8:...   
2     'dayName': 'Tomorrow', 'hours': '5:00 AM to 8:...   
3     'dayName': 'Tomo

## function for extract schedule

In [108]:
def get_schedule(schedule):
#     print(schedule)
    time = schedule.str.extract(r'(?P<Open>\d{1,2}[:]\d{2}\s\wM)').copy()
    time['Close'] = schedule.str.extract(r'(?P<to>\sto\s)(?P<Close>\d{1,2}[:]\d{2}\s\wM)')['Close'].copy()
    time['No_service'] = schedule.str.extract(r'(?P<No_service>Closed)')['No_service'].copy()
    time['Open'] = pd.to_datetime(time['Open']).dt.strftime('%H:%M')
    time['Close'] = pd.to_datetime(time['Close']).dt.strftime('%H:%M')
    filter = (time['No_service'] == 'Closed')
    time.loc[filter,('Open', 'Close')] = time.loc[filter,('Open', 'Close')].fillna('Closed')
    return time['Open'], time['Close']   

In [109]:
hours_add_time_us['Mon_open'], hours_add_time_us['Mon_close'] = get_schedule(schedule_us_split[6])
hours_add_time_us['Tue_open'], hours_add_time_us['Tue_close'] = get_schedule(schedule_us_split[0])
hours_add_time_us['Wed_open'], hours_add_time_us['Wed_close'] = get_schedule(schedule_us_split[1])
hours_add_time_us['Thu_open'], hours_add_time_us['Thu_close'] = get_schedule(schedule_us_split[2])
hours_add_time_us['Fri_open'], hours_add_time_us['Fri_close'] = get_schedule(schedule_us_split[3])
hours_add_time_us['Sat_open'], hours_add_time_us['Sat_close'] = get_schedule(schedule_us_split[4])
hours_add_time_us['Sun_open'], hours_add_time_us['Sun_close'] = get_schedule(schedule_us_split[5])
hours_add_time_us

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,...,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
0,4712,49987-272024,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",hyatt-bethesda-metro-1-bethesda-metro-center-b...,38.984316,-77.094665,1 Bethesda Metro Center,NaN,...,05:30,20:00,05:30,20:00,05:30,20:00,05:30,20:00,05:30,20:00
1,4713,743-294452,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",7200-wisconsin-ave-7200-wisconsin-ave-bethesda...,38.981224,-77.093287,7200 Wisconsin Ave,120,...,05:00,20:00,05:00,20:00,05:00,20:30,05:30,20:30,06:00,20:00
2,4714,13235-105606,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",van-ness-udc-4250-connecticut-ave-nw-van-ness-...,38.944498,-77.064207,"4250 Connecticut Ave., NW",NaN,...,05:00,20:00,05:00,20:00,05:00,20:00,06:00,18:00,06:00,18:00
3,4715,47040-256424,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",dakota-crossing-2430-market-street-ne-washingt...,38.920244,-76.952774,2430 Market Street NE,1001,...,05:30,20:00,05:30,20:00,05:30,20:00,06:00,20:00,06:00,20:00
4,4716,7764-61020,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",friendship-heights-5454-wisconsin-avenue-the-b...,38.962553,-77.086862,5454 Wisconsin Avenue,NaN,...,05:00,19:30,05:00,19:30,05:00,19:30,06:00,19:30,06:00,19:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,7530,24426-237911,US,CO,"[{'dayName': 'Today', 'hours': '4:30 AM to 9:0...",27th-avenue-loop-101-phoenix-20216-n-27th-aven...,33.670760,-112.117874,20216 N 27th Avenue,NaN,...,04:30,21:00,04:30,21:00,04:30,21:00,05:00,21:00,05:00,19:00
2819,7531,27544-245014,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 8:0...",target-phoenix-t-1141-2727-w-agua-fria-fwy-pho...,33.667883,-112.118782,2727 W Agua Fria Fwy,NaN,...,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00
2820,7532,8974-95021,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 7:3...",happy-valley-i-17-phoenix-2501-west-happy-vall...,33.711073,-112.111442,2501 West Happy Valley Road,#50-1200,...,05:00,19:30,05:00,19:30,05:00,19:30,05:00,19:30,05:30,19:30
2821,7533,9065-113971,US,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 1:3...",usaa-phoenix-campus-1-norterra-dr-phoenix-az-8...,33.720295,-112.111798,1 Norterra Dr,NaN,...,07:00,13:30,07:00,13:30,07:00,13:30,Closed,Closed,Closed,Closed


In [110]:
hours_add_time_gb['Mon_open'], hours_add_time_gb['Mon_close'] = get_schedule(schedule_gb_split[6])
hours_add_time_gb['Tue_open'], hours_add_time_gb['Tue_close'] = get_schedule(schedule_gb_split[0])
hours_add_time_gb['Wed_open'], hours_add_time_gb['Wed_close'] = get_schedule(schedule_gb_split[1])
hours_add_time_gb['Thu_open'], hours_add_time_gb['Thu_close'] = get_schedule(schedule_gb_split[2])
hours_add_time_gb['Fri_open'], hours_add_time_gb['Fri_close'] = get_schedule(schedule_gb_split[3])
hours_add_time_gb['Sat_open'], hours_add_time_gb['Sat_close'] = get_schedule(schedule_gb_split[4])
hours_add_time_gb['Sun_open'], hours_add_time_gb['Sun_close'] = get_schedule(schedule_gb_split[5])
hours_add_time_gb

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,...,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
2823,16669,9155-152277,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-south-welcome-break-m-4-motorway-westb...,51.483556,-1.557143,"M4 Motorway Westbound, Junction 14/15",Lambourne,...,06:00,22:00,06:00,22:00,06:00,22:00,07:00,22:00,07:00,22:00
2824,16670,22194-218828,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-south-welcome-break-dt-m-4-motorway-j-...,51.482387,-1.555109,M4 Motorway J14 15 membury,NaN,...,05:30,20:00,05:30,20:00,05:30,20:00,07:00,20:00,07:00,20:00
2825,16671,18362-190424,GB,LS,"[{'dayName': 'Today', 'hours': '5:30 AM to 8:0...",membury-north-welcome-break-dt-m-4-motorway-ju...,51.481264,-1.556526,"M4 Motorway, Junction 14/15",Lambourne,...,05:30,20:00,05:30,20:00,05:30,20:00,06:30,20:00,06:30,21:00
2826,16672,9136-152279,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",membury-north-welcome-break-m-4-motorway-eastb...,51.481177,-1.557422,M4 Motorway Eastbound Junction 14 15,NaN,...,06:00,22:00,06:00,22:00,06:00,22:00,07:00,22:00,07:00,22:00
2827,16673,47832-260044,GB,FR,"[{'dayName': 'Today', 'hours': '10:00 AM to 8:...",swindon-great-western-outlet-unit-51-swindon-e...,51.562617,-1.798111,Unit 51,Great Western Designer Village,...,10:00,20:00,10:00,20:00,10:00,20:00,09:00,19:00,10:00,18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,17639,21472-213065,GB,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 7:0...",rhyl-kinmel-park-eastbound-kinmel-park-a-55-ea...,53.270439,-3.519619,"Kinmel Park, A55 Eastbound",Bodelwyddon,...,07:00,19:00,07:00,19:00,07:00,19:00,07:00,19:00,08:00,20:30
3794,17640,47867-260072,GB,FR,"[{'dayName': 'Today', 'hours': '6:30 AM to 5:3...",welwyn-garden-city-howard-centre-unit-56-welwy...,51.802087,-0.204220,Unit 56,The Howard Centre,...,06:30,17:30,06:30,17:30,06:30,17:30,08:00,17:30,09:00,16:00
3795,17641,9139-152280,GB,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",birchanger-welcome-break-m-11-motorway-junctio...,51.871540,0.194940,"M11 Motorway, Junction 8","Old Dunmow Road, Bishop Stortford",...,06:00,22:00,06:00,22:00,06:00,22:00,07:00,22:00,07:00,22:00
3796,17642,27642-248279,GB,FR,"[{'dayName': 'Today', 'hours': '7:00 AM to 6:0...",bishops-stortford-jacksons-square-jacksons-squ...,51.870230,0.160930,Jacksons Square,NaN,...,07:00,18:00,07:00,18:00,07:00,18:00,07:00,18:00,10:00,17:00


## for check all null

In [111]:
print(hours_add_time_us[hours_add_time_us['Mon_open'].isnull()])
print(hours_add_time_gb[hours_add_time_gb['Sat_open'].isnull()])

       No.   storeNumber countryCode ownershipTypeCode schedule  \
811   5523    2408-13139          US                CO      NaN   
1245  5957    9609-97425          US                CO      NaN   
1497  6209      7232-567          US                CO      NaN   
1673  6385  62851-295040          US                LS      NaN   
1684  6396    9367-96488          US                CO      NaN   
1788  6500  23930-227526          US                LS      NaN   
1813  6525  26930-246154          US                CO      NaN   
1902  6614    7772-66123          US                CO      NaN   
2276  6988  59274-292333          US                LS      NaN   
2639  7351  65442-299588          US                LS      NaN   
2691  7403  67922-290584          US                LS      NaN   
2725  7437      866-1800          US                CO      NaN   

                                                   slug   latitude  \
811   pine-creek-shoppes-8958-east-96th-street-pine-...  3

# Drop/ Reorder colums

In [112]:
hours_add_time_us.columns

Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule',
       'slug', 'latitude', 'longitude', 'streetAddressLine1',
       'streetAddressLine2', 'streetAddressLine3', 'city',
       'countrySubdivisionCode', 'postalCode', 'currentTimeOffset',
       'windowsTimeZoneId', 'GMT', 'GMT_region', 'address', 'Mon_open',
       'Mon_close', 'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close',
       'Thu_open', 'Thu_close', 'Fri_open', 'Fri_close', 'Sat_open',
       'Sat_close', 'Sun_open', 'Sun_close'],
      dtype='object')

In [113]:
hours_add_time_us.drop(columns=['schedule', 'slug', 'streetAddressLine1', 'streetAddressLine2', 'streetAddressLine3', 'currentTimeOffset'], inplace = True)
hours_add_time_gb.drop(columns=['schedule', 'slug', 'streetAddressLine1', 'streetAddressLine2', 'streetAddressLine3', 'currentTimeOffset'], inplace = True)


In [114]:
column_names = ['No.','storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude', 'longitude', 'address', 'city', 'countrySubdivisionCode', 'postalCode', 'windowsTimeZoneId', 'GMT', 'GMT_region', 'Mon_open', 'Mon_close', 'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close', 'Thu_open', 'Tue_close', 'Fri_open', 'Fri_close' , 'Sat_open', 'Sat_close', 'Sun_open', 'Sun_close']
hours_add_time_us = hours_add_time_us.reindex(columns=column_names)
hours_add_time_gb = hours_add_time_gb.reindex(columns=column_names)

In [87]:
print(hours_add_time_us.columns)
print(hours_add_time_gb.columns)

Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude',
       'longitude', 'address', 'city', 'countrySubdivisionCode', 'postalCode',
       'windowsTimeZoneId', 'GMT', 'GMT_region', 'Mon_open', 'Mon_close',
       'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close', 'Thu_open',
       'Tue_close', 'Fri_open', 'Fri_close', 'Sat_open', 'Sat_close',
       'Sun_open', 'Sun_close'],
      dtype='object')
Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude',
       'longitude', 'address', 'city', 'countrySubdivisionCode', 'postalCode',
       'windowsTimeZoneId', 'GMT', 'GMT_region', 'Mon_open', 'Mon_close',
       'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close', 'Thu_open',
       'Tue_close', 'Fri_open', 'Fri_close', 'Sat_open', 'Sat_close',
       'Sun_open', 'Sun_close'],
      dtype='object')


# Merge and output

In [115]:
final = hours_add_time_us.append(hours_add_time_gb)
final

,No.,storeNumber,countryCode,ownershipTypeCode,latitude,longitude,address,city,countrySubdivisionCode,postalCode,...,Wed_open,Wed_close,Thu_open,Tue_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
0,4712,49987-272024,US,CO,38.984316,-77.094665,1 Bethesda Metro Center,bethesda,MD,208145325,...,05:30,20:00,05:30,20:00,05:30,20:00,05:30,20:00,05:30,20:00
1,4713,743-294452,US,CO,38.981224,-77.093287,7200 Wisconsin Ave|120,bethesda,MD,20814,...,05:00,20:00,05:00,20:00,05:00,20:30,05:30,20:30,06:00,20:00
2,4714,13235-105606,US,CO,38.944498,-77.064207,"4250 Connecticut Ave., NW|Van Ness Metro Building",washington,DC,200081173,...,05:00,20:00,05:00,20:00,05:00,20:00,06:00,18:00,06:00,18:00
3,4715,47040-256424,US,CO,38.920244,-76.952774,2430 Market Street NE|1001,washington,DC,200183852,...,05:30,20:00,05:30,20:00,05:30,20:00,06:00,20:00,06:00,20:00
4,4716,7764-61020,US,CO,38.962553,-77.086862,5454 Wisconsin Avenue|The Barlow Building,chevy chase,MD,208156901,...,05:00,19:30,05:00,19:30,05:00,19:30,06:00,19:30,06:00,19:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,17639,21472-213065,GB,LS,53.270439,-3.519619,"Kinmel Park, A55 Eastbound|Bodelwyddon",rhyl,WLS,LL18 5XW,...,07:00,19:00,07:00,19:00,07:00,19:00,07:00,19:00,08:00,20:30
3794,17640,47867-260072,GB,FR,51.802087,-0.204220,Unit 56|The Howard Centre,welwyn garden city,ENG,AL8 6HA,...,06:30,17:30,06:30,17:30,06:30,17:30,08:00,17:30,09:00,16:00
3795,17641,9139-152280,GB,LS,51.871540,0.194940,"M11 Motorway, Junction 8|Old Dunmow Road, Bish...",hertfordshire,ENG,CM23 5QZ,...,06:00,22:00,06:00,22:00,06:00,22:00,07:00,22:00,07:00,22:00
3796,17642,27642-248279,GB,FR,51.870230,0.160930,Jacksons Square,bishop's stortford,ENG,CM23 3AB,...,07:00,18:00,07:00,18:00,07:00,18:00,07:00,18:00,10:00,17:00


In [116]:
final.to_csv('starbucks_pd.csv', index = False)